## Estructura del Proyecto: Desarrollo Integral de un Sistema de Domótica-IoT con IA de Gemini
Este proyecto interactúa con la API de Google Gemini para dos propósitos principales:

1. **Generación de texto:** Permite obtener respuestas textuales detalladas a partir de prompts específicos, cuya finalidad es el diseño de un sistema de domótica (tanto hardware como firmware).
2. **Generación de imágenes:** Permite crear imágenes a partir de descripciones textuales, como el diseño de un logo.
El código está estructurado para facilitar la comprensión y la reutilización de las funciones.
---
## 1. Instalación de Librerías y Configuraciones Iniciales
En esta sección, instalamos las librerías necesarias y configuramos el cliente de la API de Google Gemini.

In [1]:
# Instala las librerías necesarias si aún no las tienes
#pip install -U -q google-genai
#!pip install Pillow
#!pip install python-dotenv

from google import genai
from google.genai import types
from PIL import Image
from io import BytesIO
import base64
import os # Importamos 'os' para manejar variables de entorno
from dotenv import load_dotenv

load_dotenv()  # Carga las variables del archivo .env. Colocar API Key dentro de GOOGLE_API_KEY=
api_key = os.getenv("GOOGLE_API_KEY")

client = genai.Client(api_key=api_key)


## 2. Función para Generar Prompts Estructurados
Esta función ```(generate_prompt)``` crea un prompt detallado para el modelo Gemini, permitiéndote definir el rol del modelo, la tarea, y añadir restricciones y ejemplos para guiar la respuesta.

In [2]:
def generate_prompt(role, task, constraints=None, examples=None):
    """
    Genera un prompt estructurado para el modelo Gemini.

    Argumentos:
        role (str): El rol que el modelo debe asumir (ej. "Experto en electrónica").
        task (str): La tarea específica que el modelo debe realizar.
        constraints (list, opcional): Una lista de restricciones o requisitos.
        examples (list, opcional): Una lista de ejemplos de la salida deseada.

    Retorna:
        str: El prompt completo.
    """
    prompt = f"Eres un {role}. Tu tarea es {task}. "

    if constraints:
        prompt += "\nConsidera las siguientes restricciones:\n"
        for constraint in constraints:
            prompt += f"- {constraint}\n"

    if examples:
        prompt += "\nAquí hay un ejemplo del formato de salida deseado:\n"
        for example in examples:
            prompt += f"- {example}\n"
    
    prompt += "\nPor favor, proporciona una respuesta clara y concisa."
    return prompt

## 3. Funciones para Interactuar con la API de Gemini
Aquí definimos dos funciones clave: una para generar imágenes a partir de texto y otra para generar texto a partir de texto.

**3.1. ```text_to_img``` (Generación de Imágenes)**
Esta función utiliza el modelo ```gemini-2.0-flash-preview-image-generation``` para crear una imagen basada en el prompt proporcionado. La imagen generada se guarda, en este caso, como ```iot-logo.png```.

In [3]:
def text_to_img(image_text):

    """
    Función para generar imagen a partir de texto

    Argumento:
        image_text: Prompt para generar la imagen.
    """
    
    response = client.models.generate_content(
        model="gemini-2.0-flash-preview-image-generation",
        contents=image_text,
        config=types.GenerateContentConfig(
          response_modalities=['TEXT', 'IMAGE']
        )
    )
    
    for part in response.candidates[0].content.parts:
      if part.text is not None:
        print(part.text)
      elif part.inline_data is not None:
        image = Image.open(BytesIO((part.inline_data.data)))
        image.save('iot-logo.png')
    return

**3.2. ```text_to_text``` (Generación de Texto)**
Esta función utiliza el modelo ```gemini-2.0-flash``` para generar respuestas textuales a partir de un prompt de texto.

In [4]:
def text_to_text(text_content):
    
    """
    Función para generar respuestas a partir de texto usando la IA

    Argumento:
        text_content: Prompt para generar la respuesta de texto.
    """

    response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=text_content
    )
    print(response.text)
    return

## 4. Ejemplos de Uso de las Funciones
En esta sección, demostramos cómo utilizar las funciones generate_prompt, text_to_text y text_to_img para casos de uso específicos.

### 4.1. Diseño de un Sistema de Domótica Integral (Generación de Texto)
Aquí generamos un prompt para que el modelo actúe como un "ingeniero de sistemas de domótica" y diseñe un sistema integral. La respuesta se imprimirá en la consola.

In [5]:
prompt_sistema_integral = generate_prompt(
    role="ingeniero de sistemas de domótica",
    task="diseñar un sistema de domótica integral con un ESP32 como microcontrolador principal, capaz de controlar luces, electrodomésticos y sensores del hogar. Sugiere todos los componentes electrónicos necesarios (relevadores, sensores, fuentes, protecciones), y propón una arquitectura robusta de hardware y software. Incluye opciones para la conexión a plataformas IoT compatibles con celular (como Home Assistant, Blynk, etc.). El sistema debe ser escalable y seguro.",
    constraints=[
        "El microcontrolador principal debe ser un ESP32.",
        "Debe controlar luces, electrodomésticos y sensores.",
        "Listar componentes electrónicos con su función.",
        "Describir la arquitectura de hardware (diagrama conceptual si es posible con texto).",
        "Describir la arquitectura de software (componentes clave y flujo).",
        "Mencionar al menos dos plataformas IoT compatibles con celular.",
        "El diseño debe enfatizar la escalabilidad y seguridad."
    ]
)
print("--- PROMPT: Diseño de Sistema Integral ---")
print(prompt_sistema_integral)
print("\n--- RESPUESTA: Diseño de Sistema Integral ---")
response_sistema_integral = text_to_text(prompt_sistema_integral) 

--- PROMPT: Diseño de Sistema Integral ---
Eres un ingeniero de sistemas de domótica. Tu tarea es diseñar un sistema de domótica integral con un ESP32 como microcontrolador principal, capaz de controlar luces, electrodomésticos y sensores del hogar. Sugiere todos los componentes electrónicos necesarios (relevadores, sensores, fuentes, protecciones), y propón una arquitectura robusta de hardware y software. Incluye opciones para la conexión a plataformas IoT compatibles con celular (como Home Assistant, Blynk, etc.). El sistema debe ser escalable y seguro.. 
Considera las siguientes restricciones:
- El microcontrolador principal debe ser un ESP32.
- Debe controlar luces, electrodomésticos y sensores.
- Listar componentes electrónicos con su función.
- Describir la arquitectura de hardware (diagrama conceptual si es posible con texto).
- Describir la arquitectura de software (componentes clave y flujo).
- Mencionar al menos dos plataformas IoT compatibles con celular.
- El diseño debe en

### 4.2. Diseño de un Logo de Tecnología IoT (Generación de Imagen)

En este ejemplo, creamos un prompt para que el modelo se comporte como un "diseñador gráfico" y describa un logo para nuestra empresa de tecnología IoT. Luego, este prompt se utiliza para generar la imagen de la marca.

In [6]:
prompt_logo = generate_prompt(
    role="diseñador gráfico especializado en logos de tecnología",
    task="diseñar un logo moderno y minimalista para una empresa de tecnología especializada en IoT (Internet de las cosas), con un enfoque en automatización, sensores inteligentes y conectividad en el hogar. Describe la idea del logo, los colores, las formas y el concepto detrás de él.",
    constraints=[
        "El logo debe ser moderno y minimalista.",
        "Debe representar IoT, automatización, sensores inteligentes y conectividad en el hogar.",
        "Describir la idea principal del logo.",
        "Sugerir una paleta de colores con códigos hex o descripciones.",
        "Describir las formas y elementos visuales.",
        "Explicar el concepto y significado detrás del diseño."
    ]
)
print("\n--- PROMPT: Diseño de Logo (Texto) ---")
print(prompt_logo)
text_to_img(prompt_logo)   


--- PROMPT: Diseño de Logo (Texto) ---
Eres un diseñador gráfico especializado en logos de tecnología. Tu tarea es diseñar un logo moderno y minimalista para una empresa de tecnología especializada en IoT (Internet de las cosas), con un enfoque en automatización, sensores inteligentes y conectividad en el hogar. Describe la idea del logo, los colores, las formas y el concepto detrás de él.. 
Considera las siguientes restricciones:
- El logo debe ser moderno y minimalista.
- Debe representar IoT, automatización, sensores inteligentes y conectividad en el hogar.
- Describir la idea principal del logo.
- Sugerir una paleta de colores con códigos hex o descripciones.
- Describir las formas y elementos visuales.
- Explicar el concepto y significado detrás del diseño.

Por favor, proporciona una respuesta clara y concisa.
Generaré un logo moderno y minimalista que simboliza IoT para el hogar.

**Idea Principal del Logo:** Un círculo abstracto interrumpido por líneas sutiles que convergen ha